## Задание 0


Для всех  заданий будем использовать обезличенные транзакционные банковские данные. Для этого считайте в переменные **tr_mcc_codes, tr_types, transactions и gender_train** из одноимённых таблиц из папки [data](https://drive.google.com/drive/folders/1YAMe7MiTxA-RSSd8Ex2p-L0Dspe6Gs4L). 

Для таблицы transactions используйте только первые n=1000000 строк. При чтении файлов обратите внимание на разделители внутри каждого из файлов - они могут различаться!

###  Описание данных
#### Таблица ```transactions.csv```
##### Описание
Таблица содержит историю транзакций клиентов банка за один год и три месяца.

##### Формат данных

```
customer_id,tr_datetime,mcc_code,tr_type,amount,term_id
111111,15 01:40:52,1111,1000,-5224,111111
111112,15 15:18:32,3333,2000,-100,11122233
...
```
##### Описание полей

 - ```customer_id``` — идентификатор клиента;
 - ```tr_datetime``` — день и время совершения транзакции (дни нумеруются с начала данных);
 - ```mcc_code``` — mcc-код транзакции;
 - ```tr_type``` — тип транзакции;
 - ```amount``` — сумма транзакции в условных единицах со знаком; ```+``` — начисление средств клиенту (приходная транзакция), ```-``` — списание средств (расходная транзакция);
 - ```term_id``` — идентификатор терминала;


#### Таблица ```gender_train.csv```

##### Описание
Данная таблица содержит информацию по полу для части клиентов, для которых он известен. Для остальных клиентов пол неизвестен.

##### Формат данных
```
customer_id,gender
111111,0
111112,1
...
```

##### Описание полей
 - ```customer_id``` — идентификатор клиента;
 - ```gender``` — пол клиента;

### Таблица ```tr_mcc_codes.csv```

##### Описание
Данная таблица содержит описание mcc-кодов транзакций.

##### Формат данных
```
mcc_code;mcc_description
1000;словесное описание mcc-кода 1000
2000;словесное описание mcc-кода 2000
...
```

##### Описание полей
 - ```mcc_code``` – mcc-код транзакции;
 - ```mcc_description``` — описание mcc-кода транзакции.

#### Таблица ```tr_types.csv```

##### Описание
Данная таблица содержит описание типов транзакций.

##### Формат данных
```
tr_type;tr_description
1000;словесное описание типа транзакции 1000
2000;словесное описание типа транзакции 2000
...
```

##### Описание полей
 - ```tr_type``` – тип транзакции;
 - ```tr_description``` — описание типа транзакции;

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
gender_train = pd.read_csv('/content/drive/MyDrive/MMO/gender_train.csv')
tr_mcc_codes = pd.read_csv('/content/drive/MyDrive/MMO/tr_mcc_codes.csv', delimiter = ';')
tr_types = pd.read_csv('/content/drive/MyDrive/MMO/tr_types.csv', delimiter = ';')
transactions = pd.read_csv('/content/drive/MyDrive/MMO/transactions.csv').head(1000000)


Задания 1-4 делаются без использования merge!

## Задание 1



1. Для столбца tr_type датафрейма transactions выберите произвольные 1000 строк с помощью метода sample



In [ ]:
a = transactions['tr_type'].sample(n=1000, random_state = 1) 

2. В полученной на предыдущем этапе подвыборке найдите долю транзакций (стобец tr_description в датасете tr_types), в которой содержится подстрока 'POS' или 'ATM'

In [ ]:
POS = tr_types.loc[tr_types.tr_description.str.contains('POS', na = False)] # отобрал только POS

ATM = tr_types.loc[tr_types.tr_description.str.contains('АТМ', na = False)] # отобрал только ATM

a_sort = a.value_counts(sort=False) # отобраны значения из выборки, а также их кол-во в таблице

In [ ]:
P_result = list(set(a_sort.index) & set(POS.tr_type.values)) #множество индексов, которое относится к POS & a_sort

A_result = list(set(a_sort.index) & set(ATM.tr_type.values)) #множество индексов, которое относится к ATM & a_sort

P_and_A = a_sort.index.isin(P_result + A_result) # выбираем значения, которые соответсвуют P_result и A_result

In [ ]:
result = a_sort[P_and_A].values.sum() / len(a)

print("Доля транзакций, в которой содержится подстрока POS или ATM: " + str(result))

Доля транзакций, в которой содержится подстрока POS или ATM: 0.608


## Задание 2 


1. Для столбца tr_type датафрейма transactions посчитайте частоту встречаемости всех типов транзакций tr_type в transactions.

In [ ]:
res = transactions.groupby(transactions['tr_type']).size() #частоту встречаемости всех типов транзакций  
res

tr_type
1000         8
1010    231117
1030    118975
1100     15476
1110    137658
         ...  
7074       505
7075        12
8100         2
8145         8
8146         3
Length: 72, dtype: int64

2. Выведите топ-10 транзакций по частоте встречаемости (вывести для них tr_description тоже).

In [ ]:
r = pd.DataFrame(res) 

r_s = r.sort_values(by = 0 , ascending=False).head(10) #сортируем по кол-ву транзакций

r_s_t = r_s.sort_values(by = 'tr_type') # сортируем по номеру транзакции 


In [ ]:
t = tr_types['tr_type'].isin(r_s_t.index) # выбираем из tr_types нужные индексы

m = tr_types[t].set_index('tr_type').sort_values(by = 'tr_type') # сортируем по номеру транзакции


In [ ]:
m['count'] = r_s_t[0] 

m.sort_values(by = 'count', ascending = False)

,tr_description,count
tr_type,,
1010,Покупка. POS ТУ СБ РФ,231117
2010,Выдача наличных в АТМ Сбербанк России,151166
7070,Перевод на карту (с карты) через Мобильный бан...,149006
1110,Покупка. POS ТУ Россия,137658
1030,Оплата услуги. Банкоматы СБ РФ,118975
2370,Списание с карты на карту по операции <перевод...,49830
7010,Взнос наличных через АТМ (в своем тер.банке),28972
7030,Перевод на карту (с карты) через АТМ (в предел...,26078
7071,Перевод на карту (с карты) через Мобильный бан...,15781


## Задание 3
1. В датафрейме transactions найдите клиента с максимальной суммой приходов на карту

In [ ]:
id_i = transactions[transactions['amount'] > 0].groupby('customer_id').sum('amount').idxmax()[2] #1

print("ID клиента с максимальной суммой приходов: " + str(id_i))

ID клиента с максимальной суммой приходов: 70780820


2. В датафрейме transactions найдите клиента с максимальной суммой расходов по карте


In [ ]:
id_o = transactions[transactions['amount'] < 0].groupby('customer_id').sum('amount').idxmin()[2] #2

print("ID клиента с максимальной суммой расходов: " + str(id_o))

ID клиента с максимальной суммой расходов: 70780820


3. Найдите модуль разницы для этих клиентов между суммой расходов и суммой приходов

In [ ]:
id_i_a = transactions[transactions['amount'] > 0].groupby(
    'customer_id'
    ).sum(
        'amount'
        ).sort_values(by = 'amount', ascending = False).head(1)['amount'].values[0] # взяли максимальную сумму приходов у пользователя

In [ ]:
id_o_a = transactions[transactions['amount'] < 0].groupby(
    'customer_id'
    ).sum(
        'amount'
        ).sort_values(by = 'amount').head(1)['amount'].values[0] # взяли максимальную сумму расходов у пользователя

In [ ]:
result_abs = round(abs(id_i_a + id_o_a), 2)

print("Разница между расходами и приходами: " + str(result_abs))

Разница между расходами и приходами: 1837317.98


## Задание 4
1. Найдите среднее арифметическое и медиану по amount по всем типам транзакций из топ 10 из задания 2


In [ ]:
arr_type = []
arr_mean = []
arr_median = []

for key, group in transactions[transactions.tr_type.isin(m.index)].groupby(['tr_type']):
  arr_type.append(key)
  arr_mean.append(group.amount.mean())
  arr_median.append(group.amount.median())

In [ ]:
d = {"tr_type":pd.Series(arr_type), "mean": pd.Series(arr_mean), "median": pd.Series(arr_median)}

pd.DataFrame(d)


,tr_type,mean,median
0,1010,-19784.748640,-7411.52
1,1030,-5320.980222,-2245.92
2,1100,-44061.827262,-10188.26
3,1110,-32119.330371,-11207.57
4,2010,-136077.629325,-44918.32
5,2370,-205418.249032,-44918.32
6,7010,276391.789596,112295.79
7,7030,86104.332909,13951.52
8,7070,65569.831700,11319.42
9,7071,66806.826623,3593.47


2. Найдите среднее арифметическое и медиану по amount по всем типам транзакций для клиентов из задания 3

In [ ]:
med_3 = transactions[transactions['customer_id'] == id_i].groupby('tr_type').median()['amount']

mea_3 = transactions[transactions['customer_id'] == id_i].groupby('tr_type').mean()['amount']


In [ ]:
df_mm = pd.DataFrame(med_3)
df_mm['mean'] = mea_3.values

df_mm.columns = df_mm.columns.str.replace("amount", "median")

df_mm

,median,mean
tr_type,,
2010,-3368873.660,-2.941056e+06
2330,-2245915.770,-2.382398e+06
2370,-2245915.770,-2.218505e+06
6110,10028.010,1.756293e+04
7020,10319.985,1.465129e+04
7021,28500.670,2.850067e+04
7030,8848.910,1.406196e+04
7031,12992.620,1.921036e+04
7034,14306.480,1.430648e+04


## Подготовка для заданий 5-8

*Из заданий 5-8 нужно выполнить минимум два любых*

Соедините transactions с всеми остальными таблицами (tr_mcc_codes, tr_types, gender_train). Причём с gender_train необходимо смёрджиться с помощью left join, а с оставшимися датафреймами - через inner.
После получения результата таблицы gender_train, tr_types, tr_mcc_codes можно удалить. В результате соединения датафреймов должно получиться 999584 строки.

In [ ]:
transactions = pd.merge(transactions, gender_train, how='left')
transactions = pd.merge(transactions, tr_mcc_codes, how='inner')
transactions = pd.merge(transactions, tr_types, how='inner')
transactions.shape

(999584, 9)

## Задание 5

1. Определите модуль разницы между средними тратами женщин и мужчин (трата - отрицательное значение amount).

Обратите внимание, что для вычисления модуля разности точных знаний о том, 
какой класc относится к мужчинам, а какой - к женщинам, не требуется.

In [ ]:
gender_1 = transactions[(transactions['gender'] == 0) & (transactions['amount'] < 0)]['amount'].mean() #1
gender_2 = transactions[(transactions['gender'] == 1) & (transactions['amount'] < 0)]['amount'].mean()

result_1 = abs(gender_1 - gender_2)

print("Модуль разницы между средними тратами женщин и мужчин: " + str(result_1))


Модуль разницы между средними тратами женщин и мужчин: 32718.05492022467


2. Определите модуль разницы между средними поступлениями у мужчин и женщин

In [ ]:
gender_11 = transactions[(transactions['gender'] == 0) & (transactions['amount'] > 0)]['amount'].mean() #2
gender_22 = transactions[(transactions['gender'] == 1) & (transactions['amount'] > 0)]['amount'].mean()

result_2 = abs(gender_11 - gender_22)

print("Модуль разницы между средними поступленими женщин и мужчин: " + str(result_2))

Модуль разницы между средними поступленими женщин и мужчин: 63366.57104801465


## Задание 6

1. По всем типам транзакций рассчитайте максимальную сумму прихода на карту (из строго положительных сумм по столбцу amount) отдельно для мужчин и женщин (назовите ее "max_income"). Оставьте по 10 типов транзакций для мужчин и для женщин, наименьших среди всех типов транзакций по полученным значениям "max_income". 

In [ ]:
max_income_0 = transactions[transactions['amount'] > 0][transactions['gender'] == 0][['amount', 'tr_type']].groupby('tr_type').agg('max').sort_values(by = 'amount', ascending = False)
max_income_1 = transactions[transactions['amount'] > 0][transactions['gender'] == 1][['amount', 'tr_type']].groupby('tr_type').agg('max').sort_values(by = 'amount', ascending = False)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  


In [ ]:
m_i_0 = max_income_0.tail(10)

m_i_0

,amount
tr_type,
2210,68315.82
2010,44918.32
1010,24839.83
2110,22459.16
7075,6737.75
4100,5041.86
2370,2335.75
4210,2245.92
4110,2245.92


In [ ]:
m_i_1 = max_income_1.tail(10)

m_i_1

,amount
tr_type,
7015,121279.45
2370,113194.15
2110,112295.79
2010,89836.63
7041,87590.72
6000,79280.83
8100,64682.37
1010,28803.87
4110,2245.92


2. Выделите среди них те типы транзакций, которые встречаются одновременно и у мужчин, и у женщин

In [ ]:
pd.merge(m_i_0, m_i_1, how = 'inner', left_on = 'tr_type', right_on = 'tr_type')

,amount_x,amount_y
tr_type,,
2010,44918.32,89836.63
1010,24839.83,28803.87
2110,22459.16,112295.79
2370,2335.75,113194.15
4110,2245.92,2245.92


## Задание 7

1. Найдите суммы затрат по каждой категории (mcc) для мужчин и для женщин. 

In [ ]:
spending_0 = transactions[transactions['amount'] < 0][transactions['gender'] == 0].groupby(['mcc_code', 'gender']).agg('sum').drop(columns = ['customer_id', 'tr_type'])
spending_0.head()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """Entry point for launching an IPython kernel.


,,amount
mcc_code,gender,
742,0.0,-386207.68
1711,0.0,-674897.69
1799,0.0,-261626.73
2741,0.0,-20746.42
3000,0.0,-74431645.27


In [ ]:
spending_1 = transactions[transactions['amount'] < 0][transactions['gender'] == 1].groupby(['mcc_code', 'gender']).agg('sum').drop(columns = ['customer_id', 'tr_type'])
spending_1.head()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """Entry point for launching an IPython kernel.


,,amount
mcc_code,gender,
742,1.0,-132277.93
1711,1.0,-206264.90
1731,1.0,-121542.67
1799,1.0,-1129257.68
2741,1.0,-221290.08


2. Найдите топ 10 категорий с самыми большими относительными модулями разности в тратах для разных полов (в ответе должны присутствовать описания mcc кодов).

In [ ]:
w = pd.merge(spending_0, spending_1, on = 'mcc_code', how='left')
w['abs'] = abs(w['amount_x'] - w['amount_y'])

w.drop(columns = ['amount_x', 'amount_y']).sort_values(by = 'abs', ascending = False).head(10)

,abs
mcc_code,
6011,3.085862e+09
4829,2.499153e+09
6010,2.208580e+08
5541,1.554326e+08
5511,7.352877e+07
5812,7.037856e+07
5977,4.241762e+07
5533,4.233808e+07
7995,4.079564e+07


## Задание 8

1. Из поля tr_datetime выделите час tr_hour, в который произошла транзакция, как первые 2 цифры до ":". (\**)

In [ ]:
transactions['tr_hour'] = [transactions['tr_datetime'][i].split()[1].split(':')[0] for i in range(len(transactions))]
transactions = transactions.astype({'tr_hour': np.int})
transactions.head(10)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  


,customer_id,tr_datetime,mcc_code,tr_type,amount,term_id,gender,mcc_description,tr_description,tr_hour
0,39026145,0 10:23:26,4814,1030,-2245.92,NaN,1.0,"Звонки с использованием телефонов, считывающих...",Оплата услуги. Банкоматы СБ РФ,10
1,39026145,6 07:08:31,4814,1030,-5614.79,NaN,1.0,"Звонки с использованием телефонов, считывающих...",Оплата услуги. Банкоматы СБ РФ,7
2,39026145,8 07:06:10,4814,1030,-1122.96,NaN,1.0,"Звонки с использованием телефонов, считывающих...",Оплата услуги. Банкоматы СБ РФ,7
3,39026145,11 08:49:03,4814,1030,-2245.92,NaN,1.0,"Звонки с использованием телефонов, считывающих...",Оплата услуги. Банкоматы СБ РФ,8
4,39026145,11 14:12:08,4814,1030,-2245.92,NaN,1.0,"Звонки с использованием телефонов, считывающих...",Оплата услуги. Банкоматы СБ РФ,14
5,39026145,11 14:15:30,4814,1030,-2245.92,NaN,1.0,"Звонки с использованием телефонов, считывающих...",Оплата услуги. Банкоматы СБ РФ,14
6,39026145,13 11:17:34,4814,1030,-2245.92,NaN,1.0,"Звонки с использованием телефонов, считывающих...",Оплата услуги. Банкоматы СБ РФ,11
7,39026145,18 07:39:05,4814,1030,-2245.92,NaN,1.0,"Звонки с использованием телефонов, считывающих...",Оплата услуги. Банкоматы СБ РФ,7
8,39026145,19 13:24:35,4814,1030,-449.18,NaN,1.0,"Звонки с использованием телефонов, считывающих...",Оплата услуги. Банкоматы СБ РФ,13
9,39026145,19 13:25:31,4814,1030,-1122.96,NaN,1.0,"Звонки с использованием телефонов, считывающих...",Оплата услуги. Банкоматы СБ РФ,13


2. Посчитайте количество транзакций с amount<0 в ночное время для мужчин и женщин. Ночное время - это примерно 00-06 часов. 

In [ ]:
s_0_6 = transactions[(transactions['amount'] < 0) & (transactions['tr_hour'] <= 6) & (transactions['tr_hour'] >= 0)].groupby('gender').count()['tr_hour']
s_0_6

gender
0.0    43510
1.0    46694
Name: tr_hour, dtype: int64

##Задание на защиту

Исправьте задание 7. В датафрейм добавьте ещё один столбец с указанием пола, представители которого тратят по соответствующей категории больше.

In [ ]:
fff = pd.merge(spending_0, spending_1, on = 'mcc_code', how='left')
fff['abs'] = abs(fff['amount_x'] - fff['amount_y'])

fff['who_spend_more'] = np.where(fff.amount_x > fff.amount_y, 0, 1)

fff.drop(columns = ['amount_x', 'amount_y']).sort_values(by = 'abs', ascending = False).head(10)

,abs,who_spend_more
mcc_code,,
6011,3.085862e+09,0
4829,2.499153e+09,0
6010,2.208580e+08,0
5541,1.554326e+08,0
5511,7.352877e+07,0
5812,7.037856e+07,0
5977,4.241762e+07,1
5533,4.233808e+07,0
7995,4.079564e+07,0
